In [2]:
!pip install pandas_market_calendars

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime
import pandas_market_calendars as mcal

from scipy.stats import pearsonr
# from sklearn.preprocessing import StandardScaler
from scipy.stats.mstats import winsorize

import time
import warnings

from tslearn.metrics import dtw

# Ignore all warnings
warnings.filterwarnings("ignore")

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

# pd.set_option('display.max_rows', 10)

In [2]:
class make_dataset:
    def __init__(self,df,df2):
        self.df = df
        self.df2 = df2
        self.df_dt = self.to_datetime_index(df)
        self.df2_dt = self.to_datetime_index(df2)
        self.df_dt = self.make_px_log(self.df_dt)
        self.df2_dt = self.make_px_log(self.df2_dt)

    def to_datetime_index(self, df):
        df['date'] = pd.to_datetime(df['date'])
        # 원본 df가 수정되지 않게 하기 위해 새로운 변수에 입력
        df_dt = df.set_index('date', inplace=False)
        return df_dt
    
    def make_px_log(self, df_dt):
        df_dt['LOG_CHG_PCT_1D'] = np.log(df_dt['CHG_PCT_1D']/100 + 1)
        df_px_log = df_dt[['PX_LAST', 'LOG_CHG_PCT_1D']]

        return df_px_log

In [3]:
# dataset에서 맨 앞에 NaN값 있을 경우 제거
# => def fill_date에서 씀
def get_first_valid_index(df, name='(티커 입력 바람!)'):
    index_without_nan1 = df['LOG_CHG_PCT_1D'].first_valid_index()
    index_without_nan2 = df['PX_LAST'].first_valid_index()

    # 에러 메시지
    if (index_without_nan1 is None) or (index_without_nan2 is None):
        print(f"{name}의 유효한 데이터가 없습니다.")
        return None

    # 더 늦은 index 찾기
    if index_without_nan1 >= index_without_nan2:
        first_valid_index = index_without_nan1
    else:
        first_valid_index = index_without_nan2
    print(f"{name}의 유효한 데이터 시작일은 {first_valid_index.strftime('%Y-%m-%d')}입니다.")
    return first_valid_index


def non_shared_holidays(market1, market2, start_date, end_date):
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')
    market1_trading_day = market1.schedule(start_date=start_date, end_date=end_date)
    market2_trading_day = market2.schedule(start_date=start_date, end_date=end_date)
    non_shared_holidays_list = sorted(set(market1_trading_day.index) - set(market2_trading_day.index))
    return non_shared_holidays_list 


def compare_index(ks_df, rs_df, ks_start_date,ks_end_date,rs_start_date,rs_end_date):
    if ks_start_date > rs_start_date:
        start_date = ks_start_date
    else:
        start_date = rs_start_date

    if ks_end_date > rs_end_date:
        end_date = rs_end_date
    else:
        end_date = ks_end_date

    if set(ks_df.loc[start_date:end_date].index) == set(rs_df.loc[start_date:end_date].index):
        print('데이터셋의 전처리 후 날짜가 잘 맞습니다')
        return None

    else:
        print('데이터셋의 전처리 후 날짜가 잘 맞지 않습니다!!!!!!!')
        return set(ks_df.loc[start_date:end_date].index), set(rs_df.loc[start_date:end_date].index)    
    
    
# 결측치 채우기 (원래의 중간 행 결측치, 비 공유 휴장일)
def fill_na(df):
    df['LOG_CHG_PCT_1D'] = df['LOG_CHG_PCT_1D'].fillna(0)
    df['PX_LAST'] = df['PX_LAST'].fillna(method='ffill')
    return df


def npdt(str_date):
    npdt_date = np.datetime64(str_date + 'T00:00:00.000000000')
    return npdt_date

In [4]:
class tlag_0_corr_dtw_rmse_len:
    def __init__(self, kospi_df, russell_df, ks_ticker, rs_ticker, timezone=-1):

        # 수정할 dataframe 생성
        self.ks_df = kospi_df
        self.rs_df = russell_df
        
        # ticker 받아오기
        self.ks_ticker = ks_ticker
        self.rs_ticker = rs_ticker
        
        # 기타 것들 받아오기
        self.timezone = timezone
        
        # dataframe 수정
        self.fill_date()
        
    # start, end 단어 사용
    def fill_date(self):
        # 시작일 구하기
        ks_start_date = get_first_valid_index(self.ks_df, self.ks_ticker)
        rs_start_date = get_first_valid_index(self.rs_df, self.rs_ticker)
        
        # 시작일로 자르기
        self.ks_df = self.ks_df.loc[ks_start_date:]
        self.rs_df = self.rs_df.loc[rs_start_date:]
        
        # 종료일 구하기
        ks_end_date = self.ks_df.index[-1]
        rs_end_date = self.rs_df.index[-1]
        
        # 한-미 시장의 주식을 비교한다면
        if self.timezone == -1:

            # 비 공유 휴장일 구하기
            krx = mcal.get_calendar('XKRX')
            nyse = mcal.get_calendar('NYSE')
            
            # 비 공유 휴장일 리스트 (fill_ks_index = 한국 휴장일, 미국 거래일)
            fill_ks_index = non_shared_holidays(nyse, krx, ks_start_date, ks_end_date)
            fill_rs_index = non_shared_holidays(krx, nyse, rs_start_date, rs_end_date)
            
            # 인덱스 채우기
            self.ks_df = self.ks_df.reindex(self.ks_df.index.union(fill_ks_index))
            self.rs_df = self.rs_df.reindex(self.rs_df.index.union(fill_rs_index))
            
       # 모든 거래일 '딕셔너리' 만들어놓기
        self.all_trading_day = {day: index for index, day in enumerate(sorted((set(self.ks_df.index)|set(self.rs_df.index))))}
        self.all_trading_idx = {index: day for index, day in enumerate(sorted((set(self.ks_df.index)|set(self.rs_df.index))))}     
        
        # 잘 채워졌나 인덱스 비교
        compare = compare_index(self.ks_df,self.rs_df,ks_start_date,ks_end_date,rs_start_date,rs_end_date)
        if compare is not None:
            if len(compare[0]-compare[1]) is not 0:
                fill_rs_index_2 = compare[0] - compare[1]
                print(f'{self.rs_ticker}에 {fill_rs_index_2}가 비어있습니다.')
                self.rs_df = self.rs_df.reindex(self.rs_df.index.union(fill_rs_index_2))
            if len(compare[1]-compare[0]) is not 0:
                fill_ks_index_2 = compare[1] - compare[0]
                print(f'{self.ks_ticker}에 {fill_ks_index_2}가 비어있습니다.')
                self.ks_df = self.ks_df.reindex(self.ks_df.index.union(fill_ks_index_2))
            print('맞지 않는 날짜를 채웠습니다!!!')
                                        
        self.ks_df = fill_na(self.ks_df)
        self.rs_df = fill_na(self.rs_df)
        
        # 자르기
        ks_start_idx = self.all_trading_day[ks_start_date]
        ks_end_idx = self.all_trading_day[ks_end_date]
        rs_start_idx = self.all_trading_day[rs_start_date]
        rs_end_idx = self.all_trading_day[rs_end_date]
        
        if ks_start_idx > rs_start_idx - self.timezone:
            rs_start_idx = ks_start_idx + self.timezone
        else:
            ks_start_idx = rs_start_idx - self.timezone

        if ks_end_idx + self.timezone > rs_end_idx:
            ks_end_idx = rs_end_date - self.timezone
        else:
            rs_end_idx = ks_end_idx + self.timezone
            
        ks_start_date = self.all_trading_idx[ks_start_idx]
        ks_end_date = self.all_trading_idx[ks_end_idx]
        rs_start_date = self.all_trading_idx[rs_start_idx]
        rs_end_date = self.all_trading_idx[rs_end_idx]
        
        self.ks_df = self.ks_df.loc[ks_start_date:ks_end_date]
        self.rs_df = self.rs_df.loc[rs_start_date:rs_end_date]
        
        self.ks_df['PX_LAST'] = np.array(self.ks_df['PX_LAST']) / self.ks_df['PX_LAST'][0]
        self.rs_df['PX_LAST'] = np.array(self.rs_df['PX_LAST']) / self.rs_df['PX_LAST'][0]
        
        ks_winsor = winsorize(self.ks_df['LOG_CHG_PCT_1D'], limits=(0.05, 0.05))
        rs_winsor = winsorize(self.rs_df['LOG_CHG_PCT_1D'], limits=(0.05, 0.05))

        self.ks_df['LOG_CHG_PCT_1D'] = ks_winsor
        self.rs_df['LOG_CHG_PCT_1D'] = rs_winsor
        
        print(f"각각 데이터셋의 길이는 {len(self.ks_df)}, {len(self.rs_df)} 입니다.")
        print(f"{self.ks_ticker} 데이터셋의 날짜는 {self.ks_df.index[0].strftime('%Y-%m-%d')} ~ {self.ks_df.index[-1].strftime('%Y-%m-%d')}로 설정됐습니다.")
        print(f"{self.rs_ticker} 데이터셋의 날짜는 {self.rs_df.index[0].strftime('%Y-%m-%d')} ~ {self.rs_df.index[-1].strftime('%Y-%m-%d')}로 설정됐습니다.")
        print()
        
    # first, last 단어 사용
    def get_corr_dtw_rmse_len(self):
        
        try:
            corr_r, p_value_r = pearsonr(self.ks_df['LOG_CHG_PCT_1D'], self.rs_df['LOG_CHG_PCT_1D'])
            if p_value_r > 0.05:
                corr_r = 0
        except:
            corr_r = np.nan
            
#         try:
#             rmse_r = np.sqrt(np.mean((np.array(self.ks_df['LOG_CHG_PCT_1D'])-np.array(self.rs_df['LOG_CHG_PCT_1D']))**2))
#         except:
#             rsme_r = np.nan
        
        try:
            corr_px, p_value_px = pearsonr(self.ks_df['PX_LAST'], self.rs_df['PX_LAST'])
            if p_value_px > 0.05:
                corr_px = 0
        except:
            corr_px = np.nan
              
#         try:
#             distance = dtw(np.array(self.ks_df['PX_LAST']), np.array(self.rs_df['PX_LAST']))
#         except:
#             distance = np.nan
            
#         try:
#             rmse_px = np.sqrt(np.mean((np.array(self.ks_df['PX_LAST'])-np.array(self.rs_df['PX_LAST']))**2))
#         except:
#             rmse_px = np.nan
              
#         return corr_r, rmse_r, corr_px, distance, rmse_px, len(self.ks_df), len(self.rs_df)
    
        return corr_r, corr_px, len(self.ks_df), len(self.rs_df)

In [5]:
path = '/Users/admin/Desktop/Life'

kospi_members_df = pd.read_csv(f'{path}/kospi200_sector.csv', encoding='cp949')
russell_members_df = pd.read_csv(f'{path}/russell3000_sector.csv', encoding='cp949')

# 섹터 리스트 -> kospi에 real estate 종목이 없음
sector_list = ['Health Care', 'Financials', 'Industrials', 'Information Technology', 'Consumer Discretionary', 'Real Estate', 'Energy', 'Materials', 'Communication Services', 'Consumer Staples', 'Utilities']
# sector_kospi = ['Health Care', 'Financials', 'Industrials', 'Information Technology', 'Consumer Discretionary', 'Energy', 'Materials', 'Communication Services', 'Consumer Staples', 'Utilities']
sector_kospi = ['Information Technology']

In [11]:
for sector in sector_kospi:
    
    all_pair_num = len(kospi_members_df[kospi_members_df['Sector']==sector]['Ticker_Bloomberg']) * len(russell_members_df[russell_members_df['Sector']==sector]['Ticker_Bloomberg'])
    
    i = 1
    
    result_list_for_df = []
    fail_list_for_df = []
    
    for kospi_ticker in kospi_members_df[kospi_members_df['Sector']==sector]['Ticker_Bloomberg']:
        for russell_ticker in russell_members_df[russell_members_df['Sector']==sector]['Ticker_Bloomberg']:
    
            start_time = time.time()

            df = pd.read_csv(f'{path}/datasets-total-ver23060812/datasets-KOSPI200/dataset-KOSPI200-{kospi_ticker}.csv')
            df2 = pd.read_csv(f'{path}/datasets-total-ver23060812/datasets-Russell3000/dataset-Russell3000-{russell_ticker}.csv')

#             try:
            error_num = 1
            data = make_dataset(df, df2)

            error_num = 2
            pair = tlag_0_corr_dtw_rmse_len(data.df_dt.iloc[-60:], data.df2_dt.iloc[-60:], kospi_ticker, russell_ticker, timezone = -1)

            error_num = 3
            result = pair.get_corr_dtw_rmse_len()

            end_time = time.time()
            excution_time = end_time - start_time

#             result_list = [kospi_ticker, russell_ticker, result[5], result[6], result[0], result[1], result[2], result[3], result[4]]
            result_list = [kospi_ticker, russell_ticker, result[2], result[3], result[0], result[1]]
            result_list_for_df.append(result_list)
#             print(f'({i}/{all_pair_num}) {kospi_ticker}|{russell_ticker}는 corr_r: {result[0]}, rmse_r: {result[1]} 로 결과 기록됨')
#             print(f'({i}/{all_pair_num}) {kospi_ticker}|{russell_ticker}는 corr_px: {result[2]}, dtw_px: {result[3]}, rmse_px: {result[4]} 로 결과 기록됨')
            print(f'({i}/{all_pair_num}) {kospi_ticker}|{russell_ticker}는 corr_r: {result[0]}, corr_px: {result[1]} 로 결과 기록됨')
        
            
            error_num = 0

#             except:
#                 end_time = time.time()
#                 excution_time = end_time - start_time

#                 fail_list = [kospi_ticker, russell_ticker, error_num]
#                 fail_list_for_df.append(fail_list)
#                 print(f'({i}/{all_pair_num}) {kospi_ticker}|{russell_ticker}는 에러{error_num}로 인해 실패로 기록됨')


            print(excution_time)
            print()
            print('-'*50)
            print()
            i += 1
    
#     df = pd.DataFrame(result_list_for_df, columns=['kospi_ticker', 'russell_ticker', 'ks_len', 'rs_len', 'corr_r', 'rmse_r', 'corr_px', 'distance_px', 'rmse_px'])
    df = pd.DataFrame(result_list_for_df, columns=['kospi_ticker', 'russell_ticker', 'ks_len', 'rs_len', 'corr_r', 'corr_px'])
    df2 = pd.DataFrame(fail_list_for_df, columns=['kospi_ticker', 'russell_ticker', 'error_num'])

    df.to_csv(f'detection_result/{sector}_get_corr_len60.csv', index=None)
    df2.to_csv(f'detection_result/{sector}_get_corr_len60_fail.csv', index=None)   

000660 KS의 유효한 데이터 시작일은 2023-03-08입니다.
AAPL UW의 유효한 데이터 시작일은 2023-03-08입니다.
데이터셋의 전처리 후 날짜가 잘 맞습니다
각각 데이터셋의 길이는 61, 61 입니다.
000660 KS 데이터셋의 날짜는 2023-03-09 ~ 2023-06-02로 설정됐습니다.
AAPL UW 데이터셋의 날짜는 2023-03-08 ~ 2023-06-01로 설정됐습니다.

(1/3950) 000660 KS|AAPL UW는 corr_r: 0, corr_px: 0.7025923449951039 로 결과 기록됨
1.5204331874847412

--------------------------------------------------

000660 KS의 유효한 데이터 시작일은 2023-03-08입니다.
ACN UN의 유효한 데이터 시작일은 2023-03-08입니다.
데이터셋의 전처리 후 날짜가 잘 맞습니다
각각 데이터셋의 길이는 61, 61 입니다.
000660 KS 데이터셋의 날짜는 2023-03-09 ~ 2023-06-02로 설정됐습니다.
ACN UN 데이터셋의 날짜는 2023-03-08 ~ 2023-06-01로 설정됐습니다.

(2/3950) 000660 KS|ACN UN는 corr_r: 0, corr_px: 0.7915022324775555 로 결과 기록됨
1.4968299865722656

--------------------------------------------------

000660 KS의 유효한 데이터 시작일은 2023-03-08입니다.
ADBE UW의 유효한 데이터 시작일은 2023-03-08입니다.
데이터셋의 전처리 후 날짜가 잘 맞습니다
각각 데이터셋의 길이는 61, 61 입니다.
000660 KS 데이터셋의 날짜는 2023-03-09 ~ 2023-06-02로 설정됐습니다.
ADBE UW 데이터셋의 날짜는 2023-03-08 ~ 2023-06-01로 설정됐습니다.

(3/3950) 000660 KS|A

데이터셋의 전처리 후 날짜가 잘 맞습니다
각각 데이터셋의 길이는 61, 61 입니다.
000660 KS 데이터셋의 날짜는 2023-03-09 ~ 2023-06-02로 설정됐습니다.
BR UN 데이터셋의 날짜는 2023-03-08 ~ 2023-06-01로 설정됐습니다.

(23/3950) 000660 KS|BR UN는 corr_r: 0, corr_px: 0.4401896075594046 로 결과 기록됨
1.4937379360198975

--------------------------------------------------

000660 KS의 유효한 데이터 시작일은 2023-03-08입니다.
BSY UW의 유효한 데이터 시작일은 2023-03-08입니다.
데이터셋의 전처리 후 날짜가 잘 맞습니다
각각 데이터셋의 길이는 61, 61 입니다.
000660 KS 데이터셋의 날짜는 2023-03-09 ~ 2023-06-02로 설정됐습니다.
BSY UW 데이터셋의 날짜는 2023-03-08 ~ 2023-06-01로 설정됐습니다.

(24/3950) 000660 KS|BSY UW는 corr_r: 0, corr_px: 0.8292471468471624 로 결과 기록됨
1.4966228008270264

--------------------------------------------------

000660 KS의 유효한 데이터 시작일은 2023-03-08입니다.
CCCS UW의 유효한 데이터 시작일은 2023-03-08입니다.
데이터셋의 전처리 후 날짜가 잘 맞습니다
각각 데이터셋의 길이는 61, 61 입니다.
000660 KS 데이터셋의 날짜는 2023-03-09 ~ 2023-06-02로 설정됐습니다.
CCCS UW 데이터셋의 날짜는 2023-03-08 ~ 2023-06-01로 설정됐습니다.

(25/3950) 000660 KS|CCCS UW는 corr_r: 0, corr_px: 0.881005573624373 로 결과 기록됨
1.4905400276184082

--

데이터셋의 전처리 후 날짜가 잘 맞습니다
각각 데이터셋의 길이는 61, 61 입니다.
000660 KS 데이터셋의 날짜는 2023-03-09 ~ 2023-06-02로 설정됐습니다.
DOX UW 데이터셋의 날짜는 2023-03-08 ~ 2023-06-01로 설정됐습니다.

(45/3950) 000660 KS|DOX UW는 corr_r: 0, corr_px: 0.38954485052399623 로 결과 기록됨
1.4936189651489258

--------------------------------------------------

000660 KS의 유효한 데이터 시작일은 2023-03-08입니다.
DT UN의 유효한 데이터 시작일은 2023-03-08입니다.
데이터셋의 전처리 후 날짜가 잘 맞습니다
각각 데이터셋의 길이는 61, 61 입니다.
000660 KS 데이터셋의 날짜는 2023-03-09 ~ 2023-06-02로 설정됐습니다.
DT UN 데이터셋의 날짜는 2023-03-08 ~ 2023-06-01로 설정됐습니다.

(46/3950) 000660 KS|DT UN는 corr_r: 0, corr_px: 0.8268719408900168 로 결과 기록됨
1.542891025543213

--------------------------------------------------

000660 KS의 유효한 데이터 시작일은 2023-03-08입니다.
DV UN의 유효한 데이터 시작일은 2023-03-08입니다.
데이터셋의 전처리 후 날짜가 잘 맞습니다
각각 데이터셋의 길이는 61, 61 입니다.
000660 KS 데이터셋의 날짜는 2023-03-09 ~ 2023-06-02로 설정됐습니다.
DV UN 데이터셋의 날짜는 2023-03-08 ~ 2023-06-01로 설정됐습니다.

(47/3950) 000660 KS|DV UN는 corr_r: 0, corr_px: 0.7652741765325258 로 결과 기록됨
1.485759973526001

---------

데이터셋의 전처리 후 날짜가 잘 맞습니다
각각 데이터셋의 길이는 61, 61 입니다.
000660 KS 데이터셋의 날짜는 2023-03-09 ~ 2023-06-02로 설정됐습니다.
GLW UN 데이터셋의 날짜는 2023-03-08 ~ 2023-06-01로 설정됐습니다.

(68/3950) 000660 KS|GLW UN는 corr_r: 0, corr_px: -0.5109308546454772 로 결과 기록됨
1.4871978759765625

--------------------------------------------------

000660 KS의 유효한 데이터 시작일은 2023-03-08입니다.
GPN UN의 유효한 데이터 시작일은 2023-03-08입니다.
데이터셋의 전처리 후 날짜가 잘 맞습니다
각각 데이터셋의 길이는 61, 61 입니다.
000660 KS 데이터셋의 날짜는 2023-03-09 ~ 2023-06-02로 설정됐습니다.
GPN UN 데이터셋의 날짜는 2023-03-08 ~ 2023-06-01로 설정됐습니다.

(69/3950) 000660 KS|GPN UN는 corr_r: 0, corr_px: 0 로 결과 기록됨
1.5421288013458252

--------------------------------------------------

000660 KS의 유효한 데이터 시작일은 2023-03-08입니다.
GWRE UN의 유효한 데이터 시작일은 2023-03-08입니다.
데이터셋의 전처리 후 날짜가 잘 맞습니다
각각 데이터셋의 길이는 61, 61 입니다.
000660 KS 데이터셋의 날짜는 2023-03-09 ~ 2023-06-02로 설정됐습니다.
GWRE UN 데이터셋의 날짜는 2023-03-08 ~ 2023-06-01로 설정됐습니다.

(70/3950) 000660 KS|GWRE UN는 corr_r: 0, corr_px: 0.6035812735495577 로 결과 기록됨
1.4861769676208496

---------------

데이터셋의 전처리 후 날짜가 잘 맞습니다
각각 데이터셋의 길이는 61, 61 입니다.
000660 KS 데이터셋의 날짜는 2023-03-09 ~ 2023-06-02로 설정됐습니다.
LSCC UW 데이터셋의 날짜는 2023-03-08 ~ 2023-06-01로 설정됐습니다.

(90/3950) 000660 KS|LSCC UW는 corr_r: 0, corr_px: -0.5377695522889653 로 결과 기록됨
1.503316879272461

--------------------------------------------------

000660 KS의 유효한 데이터 시작일은 2023-03-08입니다.
MA UN의 유효한 데이터 시작일은 2023-03-08입니다.
데이터셋의 전처리 후 날짜가 잘 맞습니다
각각 데이터셋의 길이는 61, 61 입니다.
000660 KS 데이터셋의 날짜는 2023-03-09 ~ 2023-06-02로 설정됐습니다.
MA UN 데이터셋의 날짜는 2023-03-08 ~ 2023-06-01로 설정됐습니다.

(91/3950) 000660 KS|MA UN는 corr_r: 0, corr_px: 0.35634187272758294 로 결과 기록됨
1.492854118347168

--------------------------------------------------

000660 KS의 유효한 데이터 시작일은 2023-03-08입니다.
MANH UW의 유효한 데이터 시작일은 2023-03-08입니다.
데이터셋의 전처리 후 날짜가 잘 맞습니다
각각 데이터셋의 길이는 61, 61 입니다.
000660 KS 데이터셋의 날짜는 2023-03-09 ~ 2023-06-02로 설정됐습니다.
MANH UW 데이터셋의 날짜는 2023-03-08 ~ 2023-06-01로 설정됐습니다.

(92/3950) 000660 KS|MANH UW는 corr_r: 0, corr_px: 0.7521852008887736 로 결과 기록됨
1.5071523189544678



...

데이터셋의 전처리 후 날짜가 잘 맞습니다
각각 데이터셋의 길이는 61, 61 입니다.
402340 KS 데이터셋의 날짜는 2023-03-09 ~ 2023-06-02로 설정됐습니다.
MIR UN 데이터셋의 날짜는 2023-03-08 ~ 2023-06-01로 설정됐습니다.

(3852/3950) 402340 KS|MIR UN는 corr_r: 0, corr_px: 0 로 결과 기록됨
1.4818048477172852

--------------------------------------------------

402340 KS의 유효한 데이터 시작일은 2023-03-08입니다.
MITK UR의 유효한 데이터 시작일은 2023-03-08입니다.
데이터셋의 전처리 후 날짜가 잘 맞습니다
각각 데이터셋의 길이는 61, 61 입니다.
402340 KS 데이터셋의 날짜는 2023-03-09 ~ 2023-06-02로 설정됐습니다.
MITK UR 데이터셋의 날짜는 2023-03-08 ~ 2023-06-01로 설정됐습니다.

(3853/3950) 402340 KS|MITK UR는 corr_r: 0, corr_px: 0.65083860376406 로 결과 기록됨
1.5201871395111084

--------------------------------------------------

402340 KS의 유효한 데이터 시작일은 2023-03-08입니다.
MLNK UN의 유효한 데이터 시작일은 2023-03-08입니다.
데이터셋의 전처리 후 날짜가 잘 맞습니다
각각 데이터셋의 길이는 61, 61 입니다.
402340 KS 데이터셋의 날짜는 2023-03-09 ~ 2023-06-02로 설정됐습니다.
MLNK UN 데이터셋의 날짜는 2023-03-08 ~ 2023-06-01로 설정됐습니다.

(3854/3950) 402340 KS|MLNK UN는 corr_r: 0, corr_px: 0.5566342119464082 로 결과 기록됨
1.4829540252685547

---------

데이터셋의 전처리 후 날짜가 잘 맞습니다
각각 데이터셋의 길이는 61, 61 입니다.
402340 KS 데이터셋의 날짜는 2023-03-09 ~ 2023-06-02로 설정됐습니다.
OSIS UW 데이터셋의 날짜는 2023-03-08 ~ 2023-06-01로 설정됐습니다.

(3874/3950) 402340 KS|OSIS UW는 corr_r: 0, corr_px: 0.8699477278467082 로 결과 기록됨
1.5522921085357666

--------------------------------------------------

402340 KS의 유효한 데이터 시작일은 2023-03-08입니다.
OSPN UR의 유효한 데이터 시작일은 2023-03-08입니다.
데이터셋의 전처리 후 날짜가 잘 맞습니다
각각 데이터셋의 길이는 61, 61 입니다.
402340 KS 데이터셋의 날짜는 2023-03-09 ~ 2023-06-02로 설정됐습니다.
OSPN UR 데이터셋의 날짜는 2023-03-08 ~ 2023-06-01로 설정됐습니다.

(3875/3950) 402340 KS|OSPN UR는 corr_r: 0, corr_px: -0.7550279374541655 로 결과 기록됨
1.4873888492584229

--------------------------------------------------

402340 KS의 유효한 데이터 시작일은 2023-03-08입니다.
OUST UN의 유효한 데이터 시작일은 2023-03-08입니다.
데이터셋의 전처리 후 날짜가 잘 맞습니다
각각 데이터셋의 길이는 61, 61 입니다.
402340 KS 데이터셋의 날짜는 2023-03-09 ~ 2023-06-02로 설정됐습니다.
OUST UN 데이터셋의 날짜는 2023-03-08 ~ 2023-06-01로 설정됐습니다.

(3876/3950) 402340 KS|OUST UN는 corr_r: 0, corr_px: -0.4984269944252612 로 결과 기록됨
1.4798

데이터셋의 전처리 후 날짜가 잘 맞습니다
각각 데이터셋의 길이는 61, 61 입니다.
402340 KS 데이터셋의 날짜는 2023-03-09 ~ 2023-06-02로 설정됐습니다.
RBBN UW 데이터셋의 날짜는 2023-03-08 ~ 2023-06-01로 설정됐습니다.

(3896/3950) 402340 KS|RBBN UW는 corr_r: 0, corr_px: -0.6508840815215434 로 결과 기록됨
1.4835879802703857

--------------------------------------------------

402340 KS의 유효한 데이터 시작일은 2023-03-08입니다.
RELY UW의 유효한 데이터 시작일은 2023-03-08입니다.
데이터셋의 전처리 후 날짜가 잘 맞습니다
각각 데이터셋의 길이는 61, 61 입니다.
402340 KS 데이터셋의 날짜는 2023-03-09 ~ 2023-06-02로 설정됐습니다.
RELY UW 데이터셋의 날짜는 2023-03-08 ~ 2023-06-01로 설정됐습니다.

(3897/3950) 402340 KS|RELY UW는 corr_r: 0, corr_px: 0.7480785577686458 로 결과 기록됨
1.480503797531128

--------------------------------------------------

402340 KS의 유효한 데이터 시작일은 2023-03-08입니다.
RGTI UR의 유효한 데이터 시작일은 2023-03-08입니다.
데이터셋의 전처리 후 날짜가 잘 맞습니다
각각 데이터셋의 길이는 61, 61 입니다.
402340 KS 데이터셋의 날짜는 2023-03-09 ~ 2023-06-02로 설정됐습니다.
RGTI UR 데이터셋의 날짜는 2023-03-08 ~ 2023-06-01로 설정됐습니다.

(3898/3950) 402340 KS|RGTI UR는 corr_r: 0, corr_px: 0.36900268692528704 로 결과 기록됨
1.53508

데이터셋의 전처리 후 날짜가 잘 맞습니다
각각 데이터셋의 길이는 61, 61 입니다.
402340 KS 데이터셋의 날짜는 2023-03-09 ~ 2023-06-02로 설정됐습니다.
SPSC UW 데이터셋의 날짜는 2023-03-08 ~ 2023-06-01로 설정됐습니다.

(3918/3950) 402340 KS|SPSC UW는 corr_r: 0, corr_px: 0.8074321788657498 로 결과 기록됨
1.5154826641082764

--------------------------------------------------

402340 KS의 유효한 데이터 시작일은 2023-03-08입니다.
SPT UR의 유효한 데이터 시작일은 2023-03-08입니다.
데이터셋의 전처리 후 날짜가 잘 맞습니다
각각 데이터셋의 길이는 61, 61 입니다.
402340 KS 데이터셋의 날짜는 2023-03-09 ~ 2023-06-02로 설정됐습니다.
SPT UR 데이터셋의 날짜는 2023-03-08 ~ 2023-06-01로 설정됐습니다.

(3919/3950) 402340 KS|SPT UR는 corr_r: 0, corr_px: -0.8362946245930143 로 결과 기록됨
1.5191500186920166

--------------------------------------------------

402340 KS의 유효한 데이터 시작일은 2023-03-08입니다.
SQSP UN의 유효한 데이터 시작일은 2023-03-08입니다.
데이터셋의 전처리 후 날짜가 잘 맞습니다
각각 데이터셋의 길이는 61, 61 입니다.
402340 KS 데이터셋의 날짜는 2023-03-09 ~ 2023-06-02로 설정됐습니다.
SQSP UN 데이터셋의 날짜는 2023-03-08 ~ 2023-06-01로 설정됐습니다.

(3920/3950) 402340 KS|SQSP UN는 corr_r: 0, corr_px: 0 로 결과 기록됨
1.479118824005127

--------

데이터셋의 전처리 후 날짜가 잘 맞습니다
각각 데이터셋의 길이는 61, 61 입니다.
402340 KS 데이터셋의 날짜는 2023-03-09 ~ 2023-06-02로 설정됐습니다.
VRRM UR 데이터셋의 날짜는 2023-03-08 ~ 2023-06-01로 설정됐습니다.

(3940/3950) 402340 KS|VRRM UR는 corr_r: 0, corr_px: 0.5536533002942858 로 결과 기록됨
1.4827377796173096

--------------------------------------------------

402340 KS의 유효한 데이터 시작일은 2023-03-08입니다.
VSH UN의 유효한 데이터 시작일은 2023-03-08입니다.
데이터셋의 전처리 후 날짜가 잘 맞습니다
각각 데이터셋의 길이는 61, 61 입니다.
402340 KS 데이터셋의 날짜는 2023-03-09 ~ 2023-06-02로 설정됐습니다.
VSH UN 데이터셋의 날짜는 2023-03-08 ~ 2023-06-01로 설정됐습니다.

(3941/3950) 402340 KS|VSH UN는 corr_r: 0, corr_px: 0.7542526259184525 로 결과 기록됨
1.482778787612915

--------------------------------------------------

402340 KS의 유효한 데이터 시작일은 2023-03-08입니다.
WEAV UN의 유효한 데이터 시작일은 2023-03-08입니다.
데이터셋의 전처리 후 날짜가 잘 맞습니다
각각 데이터셋의 길이는 61, 61 입니다.
402340 KS 데이터셋의 날짜는 2023-03-09 ~ 2023-06-02로 설정됐습니다.
WEAV UN 데이터셋의 날짜는 2023-03-08 ~ 2023-06-01로 설정됐습니다.

(3942/3950) 402340 KS|WEAV UN는 corr_r: 0, corr_px: 0.768783554259754 로 결과 기록됨
1.48599815368

In [ ]:
for sector in sector_kospi:
    
    all_pair_num = len(kospi_members_df[kospi_members_df['Sector']==sector]['Ticker_Bloomberg']) * len(russell_members_df[russell_members_df['Sector']==sector]['Ticker_Bloomberg'])
    
    i = 1
    
    result_list_for_df = []
    fail_list_for_df = []
    
    for kospi_ticker in kospi_members_df[kospi_members_df['Sector']==sector]['Ticker_Bloomberg']:
        for russell_ticker in russell_members_df[russell_members_df['Sector']==sector]['Ticker_Bloomberg']:

            df = pd.read_csv(f'{path}/datasets-total-ver23060812/datasets-KOSPI200/dataset-KOSPI200-{kospi_ticker}.csv')
            df2 = pd.read_csv(f'{path}/datasets-total-ver23060812/datasets-Russell3000/dataset-Russell3000-{russell_ticker}.csv')

            data = make_dataset(df, df2)

            pair = tlag_0_corr_dtw_rmse_len(data.df_dt.iloc[-60:], data.df2_dt.iloc[-60:], kospi_ticker, russell_ticker, timezone = -1)

            result = pair.get_corr_dtw_rmse_len()

            end_time = time.time()
            excution_time = end_time - start_time

            result_list = [kospi_ticker, russell_ticker, result[2], result[3], result[0], result[1]]
            result_list_for_df.append(result_list)
            
            print(f'({i}/{all_pair_num}) {kospi_ticker}|{russell_ticker}는 corr_r: {result[0]}, corr_px: {result[1]} 로 결과 기록됨')
            print()
            i += 1
    
#     df = pd.DataFrame(result_list_for_df, columns=['kospi_ticker', 'russell_ticker', 'ks_len', 'rs_len', 'corr_r', 'rmse_r', 'corr_px', 'distance_px', 'rmse_px'])
    df = pd.DataFrame(result_list_for_df, columns=['ticker1', 'ticker2', 'corr_r', 'corr_px'])

    df.to_csv(f'detection_result/{sector}_get_corr_len60.csv', index=None)
    df2.to_csv(f'detection_result/{sector}_get_corr_len60_fail.csv', index=None)   